In [2]:
%%javascript
/**********************************************************************************************
Known Mathjax Issue with Chrome - a rounding issue adds a border to the right of mathjax markup
https://github.com/mathjax/MathJax/issues/1300
A quick hack to fix this based on stackoverflow discussions: 
http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line
**********************************************************************************************/

$('.math>span').css("border-left-color","transparent")

<IPython.core.display.Javascript object>

In [3]:
%reload_ext autoreload
%autoreload 2

# MIDS - w261 Machine Learning At Scale


## Project 3


---
__Name:__  Hyera Moon   
__Week:__   3


# Table of Contents <a name="TOC"></a> 

1.  [HW Intructions](#1)   
2.  [HW Problems](#2)   
    3.1.   [HW3.1](#3.1)     
    3.2.   [HW3.2](#3.2)   
    3.3  [HW3.3](#3.3)   
    3.4  [HW3.4](#3.4)   
    3.5  [HW3.5](#3.5)   
    3.6  [HW3.6](#3.6)   
    3.7  [HW3.7](#3.7)   
    3.8  [HW3.8](#3.8)   
       

<a name="1"></a>
# 1 Instructions
[Back to Table of Contents](#TOC)



<a name="2"></a>
## HW3.0.
1. How do you merge  two sorted  lists/arrays of records of the form [key, value]?
1. Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
1. What is  a combiner function in the context of Hadoop? 
1. Give an example where it can be used and justify why it should be used in the context of this problem.
1. What is the Hadoop shuffle?

1) To merge two sorted lists/arrays of records of the form [key,value], we need 3 arrays: the two sorted array and an empty temporary array. Then we define 3 references (pointers) at the front of each array and move the pointers by picking the smallest key of the 2 sorted arrays and move that key with the value to the temporary array, incrementing the corresponding indices.  


2) This type of merge is used:  
- on the map side: when merging spills in a single, partitioned file  
- on the reduce side: to merge map outputs that have been copied over to the reducer  
    
    
3) A combiner is a mapper side function that allows us to combine records from mapper outputs that have the same key. They can be thought as "mini-reducers" that run in memory after the map phase. Thus, it alleviates the network traffic in the shuffling phase.  


4) An example where combiner can be used is word count of a large set of text documents. In this case, the outputs of the map task (each word in the document with a count of 1) will be very large, that is, a lot of data to transfer across the network from the mapper to the reducer. A combiner will help to aggregate the map outputs with the same key and thus will result in fewer records (e.g. 2 records with same key (word1, 1) and (word1, 1) will be combined into one single record (word1, 2)) and reduce the amount of data transfer across the network as inputs to the reducer.  


5) Hadoop shuffle is a contract between the mapper and reducer: it is the process by which the system performs the sort and transfer of the map outputs from mappers to reducers as inputs. It can be summarized by "partition, sort, combine" (in memory, on disk). The partition, sort and combine is required so that the records with same key are in the same and the inputs to reducers are sorted by key.


<a name="3.1"></a>
## HW3.1 consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:


     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:
```
Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
```
User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

In [2]:
# First download data from dropbox and create csv file
!curl 'https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0#' -L -o ccd.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 48.5M  100 48.5M    0     0  6132k      0  0:00:08  0:00:08 --:--:-- 6962k


In [6]:
# Create HDFS directories
!hdfs dfs -mkdir -p hw3/complaints

In [7]:
# Put the data into HDFS
!hdfs dfs -put ccd.csv hw3/complaints

In [9]:
%%writefile complaintCountsMapper.py
#!/usr/bin/env python
# START STUDENT CODE HW31MAPPER

import sys

sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

for line in sys.stdin:
    line = line.strip()
    complaint_ID, product, other_info = line.split(",", 2)
    if "debt" in product.lower():    
        sys.stderr.write("reporter:counter:EDA Counters,Debt,1\n") 
        print '%s\t%s' % ("debt", 1)
    elif "mortgage" in product.lower():    
        sys.stderr.write("reporter:counter:EDA Counters,Mortgage,1\n") 
        print '%s\t%s' % ("mortgage", 1)
    else:   
        sys.stderr.write("reporter:counter:EDA Counters,Other,1\n") 
        print '%s\t%s' % ("other", 1)

# END STUDENT CODE HW31MAPPER

Writing complaintCountsMapper.py


In [16]:
%%writefile complaintCountsReducer.py
#!/usr/bin/env python
# START STUDENT CODE HW31REDUCER

import sys

sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    product, count = line.split('\t', 1)

    if current_word == product:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = int(count)
        current_word = product

# Output the last word if needed
if current_word == product:
    print '%s\t%s' % (current_word, current_count)

# END STUDENT CODE HW31REDUCER

Overwriting complaintCountsReducer.py


In [11]:
!chmod a+x complaintCountsMapper.py
!chmod a+x complaintCountsReducer.py

In [17]:
# Hadoop command
# START STUDENT CODE HW31HADOOP

!hdfs dfs -rm -r HW3.1/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -files complaintCountsMapper.py,complaintCountsReducer.py \
  -mapper complaintCountsMapper.py \
  -reducer complaintCountsReducer.py \
  -input hw3/complaints/ccd.csv \
  -output HW3.1/results  \
  -numReduceTasks 1

# END STUDENT CODE HW31HADOOP

Deleted HW3.1/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob7112962568325066335.jar tmpDir=null
17/01/28 13:59:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/28 13:59:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/28 13:59:41 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/28 13:59:41 INFO mapreduce.JobSubmitter: number of splits:2
17/01/28 13:59:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0003
17/01/28 13:59:41 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0003
17/01/28 13:59:41 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0003/
17/01/28 13:59:41 INFO mapreduce.Job: Running job: job_1485626579957_0003
17/01/28 13:59:48 INFO mapreduce.Job: Job job_1485626579957_0003 running in uber mode : false
17/01/28 13:59:48 INFO mapreduce

In [19]:
%%HTML
<H1> Mapper Counters - Job Tracker UI Screenshot </H1>
<img src="mapperCounters.png">

<a name="3.2"></a>
### HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only):    
`foo foo quux labs foo bar quux`


__3.2.A__     
Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many times the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

__3.2.B__   
Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

__3.2.C__     
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Using a single reducer: 
- What are the top 50 most frequent terms in your word count analysis?    
- Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order.    
- Present bottom 10 tokens (least frequent items). 

__NOTE:__ You can use: `WORD_RE = re.compile(r"[\w']+")` to tokenize.

### 3.2.A SOLUTION

In [23]:
%%writefile onerecord.txt
foo foo quux labs foo bar quux

Writing onerecord.txt


In [28]:
%%writefile mapper3.2.A.py
#!/usr/bin/env python
# START STUDENT CODE HW32AMAPPER

import sys
import re

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

WORD_RE = re.compile(r"[\w']+")  # to tokenize

for line in sys.stdin:
    line = line.strip()
    for word in WORD_RE.findall(line.lower()):
        print '%s\t%s' % (word, 1)
        

# END STUDENT CODE HW32AMAPPER

Overwriting mapper3.2.A.py


In [24]:
%%writefile reducer3.2.A.py
#!/usr/bin/env python
# START STUDENT CODE HW32AREDUCER

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

# END STUDENT CODE HW32AREDUCER

Overwriting reducer3.2.A.py


In [25]:
!chmod a+x mapper3.2.A.py
!chmod a+x reducer3.2.A.py

In [34]:
# Hadoop command
# START STUDENT CODE HW32AHADOOP

!hdfs dfs -rm onerecord.txt # Comment when running first time
!hdfs dfs -copyFromLocal onerecord.txt
!hdfs dfs -rm -r HW3.2A/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Testing Count of Mappers and Reducers via Streaming" \
  -D mapreduce.job.maps=1 \
  -files mapper3.2.A.py,reducer3.2.A.py \
  -mapper mapper3.2.A.py \
  -reducer reducer3.2.A.py \
  -input onerecord.txt \
  -output HW3.2A/results \
  -numReduceTasks 4

# END STUDENT CODE HW32AHADOOP

Deleted onerecord.txt
Deleted HW3.2A/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob4643666819972053817.jar tmpDir=null
17/01/28 17:29:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/28 17:29:49 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/28 17:29:50 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/28 17:29:50 INFO mapreduce.JobSubmitter: number of splits:1
17/01/28 17:29:51 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0007
17/01/28 17:29:51 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0007
17/01/28 17:29:51 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0007/
17/01/28 17:29:51 INFO mapreduce.Job: Running job: job_1485626579957_0007
17/01/28 17:29:59 INFO mapreduce.Job: Job job_1485626579957_0007 running in uber mode : false
17/01/28 

In [36]:
%%HTML
# INSERT SCREENSHOT OF JOB TRACKER UI COUNTERS
<H1> Mapper and Reducer Counters - Job Tracker UI Screenshot </H1>
<img src="counters32a.png">

#### 3.2.A EXPLANATION
When the number of mappers and reducers are not specified, the counters indicate 2 calls for the mapper and 1 call for the reducer by default. The number of mappers depends on the number of blocks in hdfs when the input file is stored.

In order to obtain specifically 1 mapper and 4 reducers, we need to specify the number of mappers and reducers as option when running the hadoop streaming.

### 3.2.B SOLUTION

In [47]:
%%writefile mapper3.2.B.py
#!/usr/bin/env python
# START STUDENT CODE HW32BMAPPER

import sys
import re

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

WORD_RE = re.compile(r"[\w']+")  # to tokenize

for line in sys.stdin:
    line = line.strip()
    complaint_ID, product, sub_product, issue, other = line.split(",", 4)
    for word in WORD_RE.findall(issue.lower()):
        print '%s\t%s' % (word, 1)

# END STUDENT CODE HW32BMAPPER

Overwriting mapper3.2.B.py


In [39]:
%%writefile reducer3.2.B.py
#!/usr/bin/env python
# START STUDENT CODE HW32BREDUCER

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

# END STUDENT CODE HW32BREDUCER

Writing reducer3.2.B.py


In [40]:
!chmod a+x mapper3.2.B.py
!chmod a+x reducer3.2.B.py

In [44]:
# Hadoop command
# START STUDENT CODE HW32BHADOOP

!hdfs dfs -rm ccd.csv # Comment when running first time
!hdfs dfs -copyFromLocal ccd.csv
!hdfs dfs -rm -r HW3.2B/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Testing Count of Mappers and Reducers via Streaming" \
  -D mapreduce.job.maps=4 \
  -files mapper3.2.B.py,reducer3.2.B.py \
  -mapper mapper3.2.B.py \
  -reducer reducer3.2.B.py \
  -input ccd.csv \
  -output HW3.2B/results \
  -numReduceTasks 2

# END STUDENT CODE HW32BHADOOP

Deleted ccd.csv
Deleted HW3.2B/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob574503737770814300.jar tmpDir=null
17/01/29 11:21:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 11:21:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 11:21:12 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 11:21:13 INFO mapreduce.JobSubmitter: number of splits:4
17/01/29 11:21:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0009
17/01/29 11:21:13 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0009
17/01/29 11:21:13 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0009/
17/01/29 11:21:13 INFO mapreduce.Job: Running job: job_1485626579957_0009
17/01/29 11:21:19 INFO mapreduce.Job: Job job_1485626579957_0009 running in uber mode : false
17/01/29 11:21:1

In [45]:
# 3.2.B OUTPUT/ANSWER
# The value of my user defined Mapper Counter is 4 and the value of Reducer Counter is 2.
# These values are the same as the ones defined in the hadoop streaming.

In [46]:
%%HTML
# INSERT SCREENSHOT OF JOB TRACKER UI COUNTERS
<H1> 3.2.B Mapper and Reducer Counters - Job Tracker UI Screenshot </H1>
<img src="counters32b.png">

### 3.2.C SOLUTION

In [48]:
%%writefile mapper3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CMAPPER

import sys
import re

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

WORD_RE = re.compile(r"[\w']+")  # to tokenize

for line in sys.stdin:
    line = line.strip()
    complaint_ID, product, sub_product, issue, other = line.split(",", 4)
    for word in WORD_RE.findall(issue.lower()):
        print '%s\t%s' % (word, 1)

# END STUDENT CODE HW32CMAPPER

Writing mapper3.2.C.py


In [49]:
%%writefile combiner3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CCOMBINER

# Note: the combiner is the same as the reducer in this word count exercise
# The only difference is the user defined counter for combiners

import sys

# To count how many times combiner is called
sys.stderr.write("reporter:counter:Combiner Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

# END STUDENT CODE HW32CCOMBINER

Writing combiner3.2.C.py


In [50]:
%%writefile reducer3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CREDUCER

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

# END STUDENT CODE HW32CREDUCER

Writing reducer3.2.C.py


In [51]:
!chmod a+x mapper3.2.C.py
!chmod a+x combiner3.2.C.py
!chmod a+x reducer3.2.C.py

In [52]:
# Hadoop command
# START STUDENT CODE HW32CHADOOP

!hdfs dfs -rm ccd.csv # Comment when running first time
!hdfs dfs -copyFromLocal ccd.csv
#!hdfs dfs -rm -r HW3.2C/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Word Count with Combiners" \
  -files mapper3.2.C.py,reducer3.2.C.py,combiner3.2.C.py \
  -mapper mapper3.2.C.py \
  -reducer reducer3.2.C.py \
  -combiner combiner3.2.C.py \
  -input ccd.csv \
  -output HW3.2C/results \
  -numReduceTasks 1

# END STUDENT CODE HW32CHADOOP

Deleted ccd.csv
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob6977870791111990717.jar tmpDir=null
17/01/29 12:01:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 12:01:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 12:01:06 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 12:01:06 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 12:01:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0010
17/01/29 12:01:06 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0010
17/01/29 12:01:07 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0010/
17/01/29 12:01:07 INFO mapreduce.Job: Running job: job_1485626579957_0010
17/01/29 12:01:13 INFO mapreduce.Job: Job job_1485626579957_0010 running in uber mode : false
17/01/29 12:01:13 INFO mapreduce.Job: 

In [55]:
# 3.2.C OUTPUT/ANSWER
#The value of the unser defined counters for:
#    - mapper is 2
#    - combiner is 2
#    - reducer is 1

In [54]:
%%HTML
# INSERT SCREENSHOT OF JOB TRACKER UI COUNTERS
<H1> 3.2.C Mappers and Reducers Counter - Job tracker UI </H1>
<img src="counters32c.png">

In [81]:
%%writefile frequencies_mapper3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CFREQMAPPER

import sys
import re
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

total_count = 0
WORD_RE = re.compile(r"[\w']+")  # to tokenize

for line in sys.stdin:
    line = line.strip()
    complaint_ID, product, sub_product, issue, other = line.split(",", 4)
    for word in WORD_RE.findall(issue.lower()):
        print '%s\t%s' % (word, 1)
        total_count += 1

print '%s\t%s' % ("** totalcount", total_count)

# END STUDENT CODE HW32CFREQMAPPER

Overwriting frequencies_mapper3.2.C.py


In [82]:
%%writefile frequencies_reducer3.2.C.py
#!/usr/bin/env python
# START STUDENT CODE HW32CFREQREDUCER

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0
total_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('\t', 1)

    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            if current_word == "** totalcount":
                total_count = int(current_count)
            else:               
                print '%s\t%s\t%s' % (current_word, current_count, float(current_count)/float(total_count))
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s\t%s' % (current_word, current_count, float(current_count)/float(total_count))

# END STUDENT CODE HW32CFREQREDUCER

Overwriting frequencies_reducer3.2.C.py


In [59]:
!chmod a+x frequencies_mapper3.2.C.py
!chmod a+x frequencies_reducer3.2.C.py

In [92]:
# Hadoop command
# START STUDENT CODE HW32CFREQHADOOP

### First MR job to sort the words alphabetically
# Note: only one reducer so one partition thus no need for extra coding for total sort

!hdfs dfs -rm ccd.csv # Comment when running first time
!hdfs dfs -copyFromLocal ccd.csv
!hdfs dfs -rm -r HW3.2C2/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Word frequencies and relative frequencies sorted by word" \
  -files frequencies_mapper3.2.C.py,frequencies_reducer3.2.C.py \
  -mapper frequencies_mapper3.2.C.py \
  -reducer frequencies_reducer3.2.C.py \
  -input ccd.csv \
  -output HW3.2C2/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.2C2/results/part-0000* > 32C_non_sorted.txt


### Second MR job to sort the words by frequencies (and by words alphabetically if ties)

!hdfs dfs -rm 32C_non_sorted.txt # Comment when running first time
!hdfs dfs -copyFromLocal 32C_non_sorted.txt
!hdfs dfs -rm -r HW3.2C2sorted/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Word frequencies and relative frequencies sorted by frequencies" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input 32C_non_sorted.txt \
  -output HW3.2C2sorted/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.2C2sorted/results/part-0000* > 32C_sorted.txt

# END STUDENT CODE HW32CFREQHADOOP

Deleted ccd.csv
Deleted HW3.2C2/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob2104237707239670910.jar tmpDir=null
17/01/29 14:04:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 14:04:30 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 14:04:32 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 14:04:32 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 14:04:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0015
17/01/29 14:04:32 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0015
17/01/29 14:04:32 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0015/
17/01/29 14:04:32 INFO mapreduce.Job: Running job: job_1485626579957_0015
17/01/29 14:04:39 INFO mapreduce.Job: Job job_1485626579957_0015 running in uber mode : false
17/01/29 14:04

In [94]:
# 3.2.C OUTPUT/ANSWER
print "The top 50 most frequent terms with their frequency and relative frequency:"
!head -50 32C_sorted.txt

print "\nThe bottom 10 tokens with their frequency and relative frequency"
!tail -10 32C_sorted.txt


The top 50 most frequent terms with their frequency and relative frequency:
loan	119630	0.122011294433
modification	70487	0.0718900786653
credit	55251	0.056350798535
servicing	36767	0.0374988653551
report	34903	0.0355977615114
incorrect	29133	0.0297129068021
information	29069	0.0296476328503
on	29069	0.0296476328503
or	22533	0.0229815305314
account	20681	0.0210926655536
debt	19309	0.0196933552137
and	16448	0.0167754055909
opening	16205	0.0165275685555
club	12545	0.0127947144418
health	12545	0.0127947144418
not	12353	0.0125988925866
attempts	11848	0.0120838403114
collect	11848	0.0120838403114
cont'd	11848	0.0120838403114
owed	11848	0.0120838403114
of	10885	0.0111016713191
my	10731	0.0109446058728
deposits	10555	0.0107651025056
withdrawals	10555	0.0107651025056
problems	9484	0.00967278371986
application	8868	0.0090445219346
to	8401	0.00856822606817
unable	8178	0.00834078714266
billing	8158	0.00832038903275
other	7886	0.00804297473796
disputes	6938	0.00707610432817
communication	6920	0.00

<a name="3.2.1"></a>
### 3.2.1  
Using **2 reducers**: What are the top **50 most frequent terms** in your word count analysis? 

Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). Please **use a combiner.**

#### START STUDENT CODE HW321 (INSERT CELLS BELOW AS NEEDED)

In [388]:
%%writefile frequencies_mapper3.2.1.py
#!/usr/bin/env python

import sys
import re
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

total_count = 0
WORD_RE = re.compile(r"[\w']+")  # to tokenize

for line in sys.stdin:
    line = line.strip()
    complaint_ID, product, sub_product, issue, other = line.split(",", 4)
    for word in WORD_RE.findall(issue.lower()):
        # 2 reducers means 2 partitions, so need to group words into 2
        # Assuming all words are uniformly distributed from a to z, middle letters are m and n to divide into
        # roughly 2 equal size partitions
        if word[0] >= "n":
            group = "B"
        else:
            group = "A"
        #group = "group" + str(int(word[0] >= "n")) # if first letter before letter "n" then group0, else group 1
        print '%s\t%s\t%s' % (group, word, 1)
        
        total_count += 1

# Repeat the total count in each partition so that the relative frequencies of words within each reducer
# can be calculated with the total of words across all partitions
print '%s\t%s\t%s' % ("A", "** totalcount", total_count)
print '%s\t%s\t%s' % ("B", "** totalcount", total_count)



Overwriting frequencies_mapper3.2.1.py


In [389]:
%%writefile frequencies_combiner3.2.1.py
#!/usr/bin/env python

import sys

# To count how many times combiner is called
sys.stderr.write("reporter:counter:Combiner Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    group, word, count = line.split("\t")
   
    if current_word == word:
        current_count += int(count)
    else:
        if current_word:             
            print '%s\t%s\t%s' % (group, current_word, current_count)
        current_count = int(count)
        current_word = word

# Output the last word if needed
if current_word == word:
    print '%s\t%s\t%s' % (group, current_word, current_count)


Overwriting frequencies_combiner3.2.1.py


In [390]:
%%writefile frequencies_reducer3.2.1.py
#!/usr/bin/env python

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_word = None
current_count = 0
total_count = 0

for line in sys.stdin:
    line = line.strip()
    group, word, count = line.split("\t")   # Note: will toss out "group"
    
    if word == "** totalcount":
        total_count += int(count)
   
    elif current_word == word:
        current_count += int(count)
    else:
        if current_word:
            relative_freq = float(current_count)/float(total_count)
            print '%s\t%s\t%s' % (current_word, current_count, "{0:.5f}".format(relative_freq))
        current_count = int(count)
        current_word = word


relative_freq = float(current_count)/float(total_count)
print '%s\t%s\t%s' % (current_word, current_count, "{0:.5f}".format(relative_freq))


Overwriting frequencies_reducer3.2.1.py


In [185]:
!chmod a+x frequencies_mapper3.2.1.py
!chmod a+x frequencies_combiner3.2.1.py
!chmod a+x frequencies_reducer3.2.1.py

In [391]:
# Hadoop command

### First MR job to sort the words alphabetically
# Note: 2 reducers so need extra coding for total sort with prepending to identify 2 groups
# and specify 2 keys to sort (group and word)

!hdfs dfs -rm ccd.csv # Comment when running first time
!hdfs dfs -copyFromLocal ccd.csv
!hdfs dfs -rm -r HW3.2.1/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.num.reduce.output.key.fields=2 \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files frequencies_mapper3.2.1.py,frequencies_reducer3.2.1.py,frequencies_combiner3.2.1.py \
  -mapper frequencies_mapper3.2.1.py \
  -reducer frequencies_reducer3.2.1.py \
  -combiner frequencies_combiner3.2.1.py \
  -input ccd.csv \
  -output HW3.2.1/results \
  -numReduceTasks 2 \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner


!hdfs dfs -cat HW3.2.1/results/part-0000* > 321_non_sorted_freq.txt


### Second MR job to sort the words by frequencies (and by words alphabetically if ties)

!hdfs dfs -rm 321_non_sorted_freq.txt # Comment when running first time
!hdfs dfs -copyFromLocal 321_non_sorted_freq.txt
!hdfs dfs -rm -r HW3.2.1sorted/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Word frequencies and relative frequencies sorted by frequencies" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input 321_non_sorted_freq.txt \
  -output HW3.2.1sorted/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.2.1sorted/results/part-0000* > 321_sorted_freq.txt


Deleted ccd.csv
Deleted HW3.2.1/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob3170883263828537388.jar tmpDir=null
17/01/30 18:13:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:13:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:13:29 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/30 18:13:29 INFO mapreduce.JobSubmitter: number of splits:2
17/01/30 18:13:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0075
17/01/30 18:13:30 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0075
17/01/30 18:13:30 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0075/
17/01/30 18:13:30 INFO mapreduce.Job: Running job: job_1485626579957_0075
17/01/30 18:13:37 INFO mapreduce.Job: Job job_1485626579957_0075 running in uber mode : false
17/01/30 18:13

In [392]:
print "Using 2 Reducers"
print "The top 50 most frequent terms with their frequency and relative frequency:"
!head -50 321_sorted_freq.txt

Using 2 Reducers
The top 50 most frequent terms with their frequency and relative frequency:
loan	119630	0.12201
modification	70487	0.07189
credit	55251	0.05635
servicing	36767	0.03750
report	34903	0.03560
incorrect	29133	0.02971
information	29069	0.02965
on	29069	0.02965
or	22533	0.02298
account	20681	0.02109
debt	19309	0.01969
and	16448	0.01678
opening	16205	0.01653
club	12545	0.01279
health	12545	0.01279
not	12353	0.01260
attempts	11848	0.01208
collect	11848	0.01208
cont'd	11848	0.01208
owed	11848	0.01208
of	10885	0.01110
my	10731	0.01094
deposits	10555	0.01077
withdrawals	10555	0.01077
problems	9484	0.00967
application	8868	0.00904
to	8401	0.00857
unable	8178	0.00834
billing	8158	0.00832
other	7886	0.00804
disputes	6938	0.00708
communication	6920	0.00706
tactics	6920	0.00706
reporting	6559	0.00669
lease	6337	0.00646
the	6248	0.00637
being	5663	0.00578
by	5663	0.00578
caused	5663	0.00578
funds	5663	0.00578
low	5663	0.00578
process	5505	0.00561
disclosure	5214	0.00532
verification	52

#### END STUDENT CODE HW321

<a name="3.3"></a>
## HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
	
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset guided by the following questions:. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

#### START STUDENT CODE HW33 (INSERT CELLS BELOW AS NEEDED)

In [149]:
# First download data from dropbox and create csv file
!curl 'https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0' -L -o cart.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3377k  100 3377k    0     0  1540k      0  0:00:02  0:00:02 --:--:-- 3838k


In [163]:
%%writefile unique_mapper3.3.py
#!/usr/bin/env python

import sys
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

for line in sys.stdin:
    line = line.strip()
    products = line.split()
    for product in products:
        print '%s\t%s' % (product, 1)


Writing unique_mapper3.3.py


In [164]:
%%writefile unique_reducer3.3.py
#!/usr/bin/env python

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_product = None
total_unique_count = 0

for line in sys.stdin:
    line = line.strip()
    product, count = line.split('\t', 1)

    if current_product == product:
        continue
    else:
        if current_product:
            total_unique_count += 1  
        current_product = product

# Output the last product if needed
if current_product == product:
    total_unique_count += 1 
    
print '%s\t%s' % ("Number of unique products", total_unique_count)


Writing unique_reducer3.3.py


In [165]:
!chmod a+x unique_mapper3.3.py
!chmod a+x unique_reducer3.3.py

In [166]:
# Hadoop command

### MR job to count number of unique products
# Note: only one reducer so one partition thus no need for extra coding for total sort

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
#!hdfs dfs -rm -r HW3.3unique/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Count of unique products" \
  -files unique_mapper3.3.py,unique_reducer3.3.py \
  -mapper unique_mapper3.3.py \
  -reducer unique_reducer3.3.py \
  -input cart.txt \
  -output HW3.3unique/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.3unique/results/part-0000* > 33_unique.txt


Deleted cart.txt
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob5364230983275209578.jar tmpDir=null
17/01/29 20:20:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 20:20:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 20:20:48 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 20:20:48 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 20:20:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0039
17/01/29 20:20:49 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0039
17/01/29 20:20:49 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0039/
17/01/29 20:20:49 INFO mapreduce.Job: Running job: job_1485626579957_0039
17/01/29 20:20:56 INFO mapreduce.Job: Job job_1485626579957_0039 running in uber mode : false
17/01/29 20:20:56 INFO mapreduce.Job:

In [167]:
!head 33unique.txt

Number of unique products	12592


In [158]:
%%writefile basket_mapper3.3.py
#!/usr/bin/env python

import sys
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

basket_number = 1

for line in sys.stdin:
    line = line.strip()
    count = len(line.split())
    print '%s\t%s' % (basket_number, count)
    basket_number += 1
    

Writing basket_mapper3.3.py


In [159]:
!chmod a+x basket_mapper3.3.py

In [160]:
# Hadoop command

### MR job to count number of products per basket and sort by size
# Note: only one reducer so one partition thus no need for extra coding for total sort

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
#!hdfs dfs -rm -r HW3.3basket/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Basket size sorted by product count" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
  -files basket_mapper3.3.py \
  -mapper basket_mapper3.3.py \
  -reducer /bin/cat \
  -input cart.txt \
  -output HW3.3basket/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.3basket/results/part-0000* > 33_basket.txt

Deleted cart.txt
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8268391173331036349.jar tmpDir=null
17/01/29 20:02:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 20:02:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 20:02:11 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 20:02:11 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 20:02:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0038
17/01/29 20:02:11 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0038
17/01/29 20:02:11 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0038/
17/01/29 20:02:11 INFO mapreduce.Job: Running job: job_1485626579957_0038
17/01/29 20:02:18 INFO mapreduce.Job: Job job_1485626579957_0038 running in uber mode : false
17/01/29 20:02:18 INFO mapreduce.Job:

In [161]:
print "The largest basket is (basket number, number of products in the basket)"
!head -1 33basket.txt 

The largest basket is (basket number, number of products in the basket)
6914	37	


In [168]:
%%writefile frequencies_mapper3.3.py
#!/usr/bin/env python

import sys
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

total_count = 0

for line in sys.stdin:
    line = line.strip()
    products = line.split()
    for product in products:
        print '%s\t%s' % (product, 1)
        total_count += 1

print '%s\t%s' % ("** totalcount", total_count)


Overwriting frequencies_mapper3.3.py


In [145]:
%%writefile frequencies_reducer3.3.py
#!/usr/bin/env python

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_product = None
current_count = 0
total_count = 0

for line in sys.stdin:
    line = line.strip()
    product, count = line.split('\t', 1)

    if current_product == product:
        current_count += int(count)
    else:
        if current_product:
            if current_product == "** totalcount":  # ** totalcount will at the top of input list since sorted
                total_count = int(current_count)    # Thus, can be determined before passing over product list
            else:               
                print '%s\t%s\t%s' % (current_product, current_count, float(current_count)/float(total_count))
        current_count = int(count)
        current_product = product

# Output the last product if needed
if current_product == product:
    print '%s\t%s\t%s' % (current_product, current_count, float(current_count)/float(total_count))


Writing frequencies_reducer3.3.py


In [146]:
!chmod a+x frequencies_mapper3.3.py
!chmod a+x frequencies_reducer3.3.py

In [155]:
# Hadoop command

### First MR job to sort the products alphabetically
# Note: only one reducer so one partition thus no need for extra coding for total sort

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
!hdfs dfs -rm -r HW3.3/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Product frequencies and relative frequencies sorted by product" \
  -files frequencies_mapper3.3.py,frequencies_reducer3.3.py \
  -mapper frequencies_mapper3.3.py \
  -reducer frequencies_reducer3.3.py \
  -input cart.txt \
  -output HW3.3/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.3/results/part-0000* > 33_non_sorted_freq.txt


### Second MR job to sort the products by frequencies (and by product alphabetically if ties)

#!hdfs dfs -rm 33_non_sorted_freq.txt # Comment when running first time
!hdfs dfs -copyFromLocal 33_non_sorted_freq.txt
!hdfs dfs -rm -r HW3.3sorted/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Product sort by frequencies" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input 33_non_sorted_freq.txt \
  -output HW3.3sorted/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.3sorted/results/part-0000* > 33_sorted_freq.txt

Deleted cart.txt
Deleted HW3.3/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob7191364961557746772.jar tmpDir=null
17/01/29 19:16:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 19:16:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/29 19:16:59 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/29 19:17:00 INFO mapreduce.JobSubmitter: number of splits:2
17/01/29 19:17:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0036
17/01/29 19:17:01 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0036
17/01/29 19:17:01 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0036/
17/01/29 19:17:01 INFO mapreduce.Job: Running job: job_1485626579957_0036
17/01/29 19:17:07 INFO mapreduce.Job: Job job_1485626579957_0036 running in uber mode : false
17/01/29 19:17:

In [156]:
print "The top 50 products with highest count with frequency and relative frequency:"
!head -50 33sorted_freq.txt

The top 50 products with highest count with frequency and relative frequency:
DAI62779	6667	0.0175067747831
FRO40251	3881	0.010191059387
ELE17451	3875	0.0101753040775
GRO73461	3602	0.00945843749344
SNA80324	3044	0.00799319370628
ELE32164	2851	0.0074863979161
DAI75645	2736	0.00718442114993
SNA45677	2455	0.0064465474865
FRO31317	2330	0.0061183118711
DAI85309	2293	0.00602115412894
ELE26917	2292	0.00601852824402
FRO80039	2233	0.00586360103355
GRO21487	2115	0.00555374661261
SNA99873	2083	0.00546971829507
GRO59710	2004	0.00526227338613
GRO71621	1920	0.00504169905258
FRO85978	1918	0.00503644728273
GRO30386	1840	0.00483162825872
ELE74009	1816	0.00476860702057
GRO56726	1784	0.00468457870302
DAI63921	1773	0.00465569396887
GRO46854	1756	0.00461105392517
ELE66600	1713	0.00449814087347
DAI83733	1712	0.00449551498855
FRO32293	1702	0.00446925613932
ELE66810	1697	0.0044561267147
SNA55762	1646	0.00432220658362
DAI22177	1627	0.00427231477008
FRO78087	1531	0.00402022981745
ELE99737	1516	0.0039808415436
E

#### END STUDENT CODE HW33

<a name="3.3.1"></a>
## HW3.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

#### START STUDENT CODE HW331 (INSERT CELLS BELOW AS NEEDED)

In [393]:
%%writefile unique_mapper3.3.1.py
#!/usr/bin/env python

import sys
    
# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

for line in sys.stdin:
    line = line.strip()
    products = line.split()
    for product in products:
        if product[0] <= "m":
            group = "A"
        else:
            group = "B"
        print '%s\t%s\t%s' % (group, product, 1)


Writing unique_mapper3.3.1.py


In [394]:
%%writefile unique_reducer3.3.1.py
#!/usr/bin/env python

import sys

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_product = None
total_unique_count = 0

for line in sys.stdin:
    line = line.strip()
    group, product, count = line.split('\t', 2)

    if current_product == product:
        continue
    else:
        if current_product:
            total_unique_count += 1  
        current_product = product

# Output the last product if needed
if current_product == product:
    total_unique_count += 1 
    
print '%s\t%s' % ("Number of unique products", total_unique_count)


Writing unique_reducer3.3.1.py


In [395]:
!chmod a+x unique_mapper3.3.1.py
!chmod a+x unique_reducer3.3.1.py

In [396]:
# Hadoop command

### MR job to count number of unique products
# Note: 2 reducers so need for extra coding for total sort

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
#!hdfs dfs -rm -r HW3.31unique/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Count of unique products" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=2 \
  -D stream.num.reduce.output.key.fields=2 \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -files unique_mapper3.3.1.py,unique_reducer3.3.1.py \
  -mapper unique_mapper3.3.1.py \
  -reducer unique_reducer3.3.1.py \
  -input cart.txt \
  -output HW3.3.1unique/results \
  -numReduceTasks 2

!hdfs dfs -cat HW3.3.1unique/results/part-0000* > 331_unique.txt


Deleted cart.txt
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8119649920354346256.jar tmpDir=null
17/01/30 18:23:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:23:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:23:39 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/30 18:23:39 INFO mapreduce.JobSubmitter: number of splits:2
17/01/30 18:23:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0077
17/01/30 18:23:40 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0077
17/01/30 18:23:40 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0077/
17/01/30 18:23:40 INFO mapreduce.Job: Running job: job_1485626579957_0077
17/01/30 18:23:46 INFO mapreduce.Job: Job job_1485626579957_0077 running in uber mode : false
17/01/30 18:23:46 INFO mapreduce.Job:

In [398]:
!head 331_unique.txt
# 2 partitions from 2 reducers - thus need to sum the 2 results: 6308 + 6284 = 

Number of unique products	6308	
Number of unique products	6284	


#### END STUDENT CODE HW331

<a name="3.4"></a>
## HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

#### START STUDENT CODE HW34 (INSERT CELLS BELOW AS NEEDED)

In [202]:
%%writefile mapper3.4.py
#!/usr/bin/env python

import sys

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

total_baskets = 0

for line in sys.stdin:
    line = line.strip()
    products = sorted(line.split())  #sorted to get same order of products in a pair
    total_baskets += 1
    
    for i in range(len(products)):
        for j in range(i+1, len(products)):
            print '%s\t%s' % (products[i]+','+products[j], 1)

print '%s\t%s' % ("** baskets", total_baskets)

Overwriting mapper3.4.py


In [281]:
%%writefile reducer3.4.py
#!/usr/bin/env python

import sys

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

current_pair = None
current_count = 0
total_baskets = 0

for line in sys.stdin:
    line = line.strip()
    pair, count = line.split("\t")
    
    if pair == "** baskets":
        total_baskets += int(count)
   
    elif current_pair == pair:
        current_count += int(count)
    else:
        if current_pair:
            support = float(current_count)/float(total_baskets)
            if current_count >= 100:
                print '%s\t%s\t%s\t%s' % (current_pair.split(",")[0], current_pair.split(",")[1], \
                                      current_count, "{0:.5f}".format(support))
        current_count = int(count)
        current_pair = pair


support = float(current_count)/float(total_baskets)
if current_count >= 100:
    print '%s\t%s\t%s\t%s' % (current_pair.split(",")[0], current_pair.split(",")[1], \
                          current_count, "{0:.5f}".format(support))


Overwriting reducer3.4.py


In [198]:
!chmod a+x mapper3.4.py
!chmod a+x reducer3.4.py

In [402]:
#Hadoop command

import time
start_time = time.time()

### First MR job to sort the pairs alphabetically

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
!hdfs dfs -rm -r HW3.4/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Pair sort" \
  -files mapper3.4.py,reducer3.4.py \
  -mapper mapper3.4.py \
  -reducer reducer3.4.py \
  -input cart.txt \
  -output HW3.4/results \
  -numReduceTasks 1

!hdfs dfs -cat HW3.4/results/part-0000* > 34_pairs.txt


### Second MR job to sort by the pairs by frequencies and if ties by items (left to right)

!hdfs dfs -rm 34_pairs.txt # Comment when running first time
!hdfs dfs -copyFromLocal 34_pairs.txt
!hdfs dfs -rm -r HW3.4sorted/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Pair sort by frequencies" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr -k1,1 -k2,2" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input 34_pairs.txt \
  -output HW3.4sorted/results \
  -numReduceTasks 1

stop_time = time.time()
print 'total run time: %0.2f seconds' %(stop_time - start_time)

!hdfs dfs -cat HW3.4sorted/results/part-0000* > 34_pairs_sorted.txt

Deleted cart.txt
Deleted HW3.4/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob1375595930445120434.jar tmpDir=null
17/01/30 18:37:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:37:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 18:37:40 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/30 18:37:40 INFO mapreduce.JobSubmitter: number of splits:2
17/01/30 18:37:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0078
17/01/30 18:37:41 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0078
17/01/30 18:37:41 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0078/
17/01/30 18:37:41 INFO mapreduce.Job: Running job: job_1485626579957_0078
17/01/30 18:37:48 INFO mapreduce.Job: Job job_1485626579957_0078 running in uber mode : false
17/01/30 18:37:

In [213]:
print "The top 50 product pairs with corresponding support count (frequency) and support (relative frequency)"
!head -50 34_pairs_sorted.txt

The top 50 product pairs with corresponding support count (frequency) and support (relative frequency)
DAI62779	ELE17451	1592	0.05119
FRO40251	SNA80324	1412	0.04540
DAI75645	FRO40251	1254	0.04032
FRO40251	GRO85051	1213	0.03900
DAI62779	GRO73461	1139	0.03662
DAI75645	SNA80324	1130	0.03633
DAI62779	FRO40251	1070	0.03440
DAI62779	SNA80324	923	0.02968
DAI62779	DAI85309	918	0.02952
ELE32164	GRO59710	911	0.02929
DAI62779	DAI75645	882	0.02836
FRO40251	GRO73461	882	0.02836
DAI62779	ELE92920	877	0.02820
FRO40251	FRO92469	835	0.02685
DAI62779	ELE32164	832	0.02675
DAI75645	GRO73461	712	0.02289
DAI43223	ELE32164	711	0.02286
DAI62779	GRO30386	709	0.02280
ELE17451	FRO40251	697	0.02241
DAI85309	ELE99737	659	0.02119
DAI62779	ELE26917	650	0.02090
GRO21487	GRO73461	631	0.02029
DAI62779	SNA45677	604	0.01942
ELE17451	SNA80324	597	0.01920
DAI62779	GRO71621	595	0.01913
DAI62779	SNA55762	593	0.01907
DAI62779	DAI83733	586	0.01884
ELE17451	GRO73461	580	0.01865
GRO73461	SNA80324	562	0.01807
DAI62779	GRO59710	56

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)


Computation setup is as follow:  
- single computer  
- quad-core  
- 2 consecutive MR jobs:  
    * First one to sort the pairs by items pairs (alphabetically)  
    * Second one to sort the pairs by support count  
- 2 mappers in 1st Map reduce job and 2 mappers in the 2nd Map reduce job for a total of 4 mappers  
- 1 reducer in 1st Map reduce job and 1 reducer in the 2nd Map reduce job for a total of 2 reducers  

For the first MR job (sort pairs based on the items), user defined Counters have been defined in the mapper and reducer to count how many times each is called. 
Please see report from screenshot below.

In [214]:
%%HTML
<H1> User-defined Counters for Mappers and Reducers in First MR Job- Job Tracker UI </H1>
<img src="counters34.png">

For the second MR job (re-sort the outputs from first MR job by support count), identity mapper and reducer were used. The counters are shown in the notebook when streaming under the section "Job Counters": there are 2 mappers and 1 reducer.

#### END STUDENT CODE HW34

<a name="3.5"></a>
## HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs


#### START STUDENT CODE HW35 (INSERT CELLS BELOW AS NEEDED)

In [406]:
%%writefile mapper3.5.py
#!/usr/bin/env python

import sys
from collections import defaultdict

# To count how many times mapper is called
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

basket_stripe = defaultdict(int)
total_baskets = 0

for line in sys.stdin:
    total_baskets += 1
    
    line = line.strip()
    products = sorted(line.split())  #sorted to get same order of products in a pair
    
    for i in range(len(products)-1):
        stripe = defaultdict(int)
        for j in range(i+1, len(products)):
            stripe[products[j]] += 1
        stripe = dict(stripe)  # Converted to dictionary format so the string format can be 
                               # reconverted into a dictionary in the reducer    
        print '%s\t%s' % (products[i], stripe)


basket_stripe["all"] = total_baskets
basket_stripe = dict(basket_stripe)

print '%s\t%s' % ("** baskets", basket_stripe)

Overwriting mapper3.5.py


In [411]:
%%writefile reducer3.5.py
#!/usr/bin/env python

import sys
import ast

# To count how many times reducer is called
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")
sys.stderr.write("reporter:status:processing message...\n")

# initiate current keys, count
current_item = None
current_count = {}
total_baskets = 0

for line in sys.stdin:
    item, stripe = line.split('\t')    
    stripe = ast.literal_eval(stripe)  # convert string input into a dictionary
    
    if item == '** baskets': 
        total_baskets += int(stripe["all"])
    
    elif item == current_item:
        for item2, count in stripe.items():
            current_count[item2] = current_count.get(item2, 0) + int(count)
    
    else:
        if current_item:
            for item2, count in current_count.items():
                if count >= 100:
                    support = float(count)/float(total_baskets)
                    print '%s\t%s\t%s\t%s' % (current_item, item2, count, "{0:.5f}".format(support))
        
        current_item = item
        current_count = stripe

if current_item == item:        
    for item2, count in current_count.items():
        if count >= 100:
            support = float(count)/float(total_baskets)
            print '%s\t%s\t%s\t%s' % (current_item, item2, count, "{0:.5f}".format(support))


Overwriting reducer3.5.py


In [316]:
!chmod a+x mapper3.5.py
!chmod a+x reducer3.5.py

In [412]:
#Hadoop command

import time
start_time = time.time()

### First MR job to sort the first item alphabetically

!hdfs dfs -rm cart.txt # Comment when running first time
!hdfs dfs -copyFromLocal cart.txt
!hdfs dfs -rm -r HW3.5/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Sort with stripes" \
  -files mapper3.5.py,reducer3.5.py \
  -mapper mapper3.5.py \
  -reducer reducer3.5.py \
  -input cart.txt \
  -output HW3.5/results \
  -numReduceTasks 1
    
!hdfs dfs -cat HW3.5/results/part-0000* > 35_stripes.txt


### Second MR job to sort by the pairs by frequencies and if ties by items (left to right)

!hdfs dfs -rm 35_stripes.txt # Comment when running first time
!hdfs dfs -copyFromLocal 35_stripes.txt
!hdfs dfs -rm -r HW3.5sorted/results # Comment only when running first time

!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.8.0.jar \
  -D mapreduce.job.name="Sort by frequencies" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D stream.num.map.output.key.fields=3 \
  -D stream.map.output.key.field.separator="\t" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr -k1,1 -k2,2" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input 35_stripes.txt \
  -output HW3.5sorted/results \
  -numReduceTasks 1

stop_time = time.time()
print 'total run time: %0.2f seconds' %(stop_time - start_time)

!hdfs dfs -cat HW3.5sorted/results/part-0000* > 35_stripes_sorted.txt

Deleted cart.txt
Deleted HW3.5/results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8372009891191061172.jar tmpDir=null
17/01/30 20:56:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 20:56:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/01/30 20:56:56 INFO mapred.FileInputFormat: Total input paths to process : 1
17/01/30 20:56:56 INFO mapreduce.JobSubmitter: number of splits:2
17/01/30 20:56:57 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1485626579957_0086
17/01/30 20:56:57 INFO impl.YarnClientImpl: Submitted application application_1485626579957_0086
17/01/30 20:56:57 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1485626579957_0086/
17/01/30 20:56:57 INFO mapreduce.Job: Running job: job_1485626579957_0086
17/01/30 20:57:04 INFO mapreduce.Job: Job job_1485626579957_0086 running in uber mode : false
17/01/30 20:57:

In [379]:
print "Stripes"
print "The top 50 product pairs with corresponding support count (frequency) and support (relative frequency)"
!head -50 35_stripes_sorted.txt

Stripes
The top 50 product pairs with corresponding support count (frequency) and support (relative frequency)
DAI62779	ELE17451	1592	0.05119
FRO40251	SNA80324	1412	0.04540
DAI75645	FRO40251	1254	0.04032
FRO40251	GRO85051	1213	0.03900
DAI62779	GRO73461	1139	0.03662
DAI75645	SNA80324	1130	0.03633
DAI62779	FRO40251	1070	0.03440
DAI62779	SNA80324	923	0.02968
DAI62779	DAI85309	918	0.02952
ELE32164	GRO59710	911	0.02929
DAI62779	DAI75645	882	0.02836
FRO40251	GRO73461	882	0.02836
DAI62779	ELE92920	877	0.02820
FRO40251	FRO92469	835	0.02685
DAI62779	ELE32164	832	0.02675
DAI75645	GRO73461	712	0.02289
DAI43223	ELE32164	711	0.02286
DAI62779	GRO30386	709	0.02280
ELE17451	FRO40251	697	0.02241
DAI85309	ELE99737	659	0.02119
DAI62779	ELE26917	650	0.02090
GRO21487	GRO73461	631	0.02029
DAI62779	SNA45677	604	0.01942
ELE17451	SNA80324	597	0.01920
DAI62779	GRO71621	595	0.01913
DAI62779	SNA55762	593	0.01907
DAI62779	DAI83733	586	0.01884
ELE17451	GRO73461	580	0.01865
GRO73461	SNA80324	562	0.01807
DAI62779	GRO

Computation setup is as follow:  
- single computer  
- quad-core  
- 2 consecutive MR jobs:  
    * First one to sort the outputs of key item and stripes by key items (alphabetically)  
    * Second one to sort the item pairs by support count  
- 2 mappers in 1st Map reduce job and 2 mappers in the 2nd Map reduce job for a total of 4 mappers  
- 1 reducer in 1st Map reduce job and 1 reducer in the 2nd Map reduce job for a total of 2 reducers  

For the first MR job (sort pairs based on the items), user defined Counters have been defined in the mapper and reducer to count how many times each is called. 
Please see report from screenshot below.

In [381]:
%%HTML
<H1> User-defined Counters for Mappers and Reducers in First MR Job- Job Tracker UI </H1>
<img src="counters35.png">

For the second MR job (re-sort the outputs from first MR job by support count), identity mapper and reducer were used. The counters are shown in the notebook when streaming under the section "Job Counters": there are 2 mappers and 1 reducer.

For implementing pairs and stripes, the framework for both cases is similar: one map reduce job with 2 mappers and 1 reducer for sorting the outputs by item name alphabetically and a second map reduce job with again 2 mappers and 1 reducer for sorting the outputs by frequency.

I was expecting a faster overall map reduce job for the stripes implementation since less data were to be transferred over the network from mapper to reducer. However the stripes(100 secs) tooks 15 seconds longer than the pairs (84 secs). The conversion of the string from reducer inputs into a dictionary using ast.literal.eval has added processing time in the stripes reducer.